In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import pandas as pd
import numpy as np
from datetime import datetime
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pylab import rcParams
from sklearn.model_selection import TimeSeriesSplit
import holidays
from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from prophet.plot import plot_cross_validation_metric
from statsmodels.tools.eval_measures import rmse
from prophet.plot import add_changepoints_to_plot
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt

In [3]:
df = pd.read_csv('/content/drive/Othercomputers/My Mac/Final Project/Time-Series-Forecasting/data/cleaned_03.csv')

In [4]:
df.head()

,order_id,date,payment,item_code,quantity,gift_quantity,unit_price_vat_excl,unit_cogs,unit_rrp_vat_excl,CreatedAt,...,year,week_of_year,price_difference,morning,afternoon,evening,total_sales_country_product,avg_sales_country_product,total_sales_country_product_month,avg_sales_country_product_day
0,2000093387,2020-04-24,GOPAY_CARD,S101,1,0.0,3.51,0.00,3.50,2020-04-24 21:50:00,...,2020,17,0.01,0,0,1,215037.02,1.61,14268.25,1.56
1,2000093387,2020-04-24,GOPAY_CARD,S113,1,0.0,-0.71,0.00,-0.72,2020-04-24 21:50:00,...,2020,17,0.01,0,0,1,215037.02,1.61,14268.25,1.56
2,2000093388,2020-04-24,COD,ZB00089178,1,0.0,5.83,2.21,6.47,2020-04-24 21:45:00,...,2020,17,-0.64,0,0,1,9275270.79,38.07,526599.68,38.15
3,2000093388,2020-04-24,COD,ZB00138060,1,0.0,5.83,2.21,6.47,2020-04-24 21:45:00,...,2020,17,-0.64,0,0,1,9275270.79,38.07,526599.68,38.15
4,2000093388,2020-04-24,COD,ZB00015664,1,0.0,19.38,18.73,32.02,2020-04-24 21:45:00,...,2020,17,-12.64,0,0,1,1749344.59,25.37,118211.77,25.31


In [5]:
df1=df.round(2)
columns_to_round = ['gross_revenue','profit']
df1 = df.copy()
for col in columns_to_round:
    df1[col] = df[col].round(2)
df1.head()

,order_id,date,payment,item_code,quantity,gift_quantity,unit_price_vat_excl,unit_cogs,unit_rrp_vat_excl,CreatedAt,...,year,week_of_year,price_difference,morning,afternoon,evening,total_sales_country_product,avg_sales_country_product,total_sales_country_product_month,avg_sales_country_product_day
0,2000093387,2020-04-24,GOPAY_CARD,S101,1,0.0,3.51,0.00,3.50,2020-04-24 21:50:00,...,2020,17,0.01,0,0,1,215037.02,1.61,14268.25,1.56
1,2000093387,2020-04-24,GOPAY_CARD,S113,1,0.0,-0.71,0.00,-0.72,2020-04-24 21:50:00,...,2020,17,0.01,0,0,1,215037.02,1.61,14268.25,1.56
2,2000093388,2020-04-24,COD,ZB00089178,1,0.0,5.83,2.21,6.47,2020-04-24 21:45:00,...,2020,17,-0.64,0,0,1,9275270.79,38.07,526599.68,38.15
3,2000093388,2020-04-24,COD,ZB00138060,1,0.0,5.83,2.21,6.47,2020-04-24 21:45:00,...,2020,17,-0.64,0,0,1,9275270.79,38.07,526599.68,38.15
4,2000093388,2020-04-24,COD,ZB00015664,1,0.0,19.38,18.73,32.02,2020-04-24 21:45:00,...,2020,17,-12.64,0,0,1,1749344.59,25.37,118211.77,25.31


In [13]:
# Convert 'date' column to datetime
df['date'] = pd.to_datetime(df['date'])

df_countries = df.groupby(['date', 'country'])['gross_revenue'].sum().reset_index()
df_countries_sales = df_countries.pivot_table(index='date', columns='country', values='gross_revenue', fill_value=0)


In [14]:
# Split the data into training and testing sets
train = df_countries_sales.loc['2019-06-01':'2021-10-31']
test = df_countries_sales.loc['2021-11-01':'2022-01-14']

for country in train.columns:
    # Prepare the dataset for Prophet
    country_train = train[[country]].reset_index()
    country_train.columns = ['ds', 'y']
    
    # Fit the Prophet model and make a forecast
    model = Prophet()
    model.fit(country_train)
    future = model.make_future_dataframe(periods=14, freq='D')
    country_forecast = model.predict(future)

    # Evaluate the model on the test data
    test_forecast = country_forecast.loc[(country_forecast['ds'] >= '2021-11-01') & (country_forecast['ds'] <= '2022-01-14')]
    actual = test[[country]].reset_index().iloc[:len(test_forecast)]
    actual.columns = ['ds', 'y']
    
    mae = mean_absolute_error(actual['y'], test_forecast['yhat'])
    mse = mean_squared_error(actual['y'], test_forecast['yhat'])
    rmse = sqrt(mse)

    print(f"Country: {country}")
    print(f"MAE: {mae:.2f}")
    print(f"MSE: {mse:.2f}")
    print(f"RMSE: {rmse:.2f}")

    # Plot the forecast
    fig = model.plot(country_forecast)
    
    # Add actual sales to the plot
    plt.plot(actual['ds'], actual['y'], 'r.-', label='Actual sales')
    
    plt.title(f"Forecast and Actual Sales for Country {country}")
    plt.xlabel('Date')
    plt.ylabel('Sales')
    plt.legend()
    plt.show()


Output hidden; open in https://colab.research.google.com to view.